## Libs

In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

## Charger la donnée

In [2]:
os.chdir('tp_nlp/txts_2')

data = []
dataframe_texte = pd.DataFrame()
for file in os.listdir() : 
    num = int(file.split('.')[0])

    with open(file, 'r') as file:
        contenu = file.read()
        data.append(
            {
             'num_fichier' : num,
             'txt' : contenu

            })
textes = pd.DataFrame(data)

## Extraction des phrases 

In [3]:
textes['phrases'] =  textes['txt'].apply(lambda txt : nltk.sent_tokenize(txt, language='english') )

## TF-IDF

In [4]:
def get_tfidf_matrix(phrases):
    vectorizer = TfidfVectorizer(stop_words='english')
    return vectorizer,vectorizer.fit_transform(phrases)

textes['tfidf_matrix'] = textes['phrases'].apply(get_tfidf_matrix)

## Scoring

In [5]:
textes['scores'] = textes['tfidf_matrix'].apply(lambda tfidf_matrix : np.array(tfidf_matrix[1].sum(axis=1)).flatten() )

## Colonne résumé 

In [6]:
scores = textes['scores'].to_list()
textes_sentences = textes['phrases'].to_list()
textes['resumé'] = [ sentences[score.argmax()] for sentences , score in zip(textes_sentences,scores)]

## Mot le plus fréquent 

In [7]:
def get_best_word (ele):

    vectorizer = ele[0]
    tfidf_matrix = ele[1]

    feature_names = vectorizer.get_feature_names_out()
    word_counts = np.asarray(tfidf_matrix.sum(axis=0)).flatten() 
    word_freq = dict(zip(feature_names, word_counts))
    sorted_word_freq = sorted(word_freq.items(), key=lambda item: item[1], reverse=True)
    return sorted_word_freq

textes['count_word'] = textes['tfidf_matrix'].apply(get_best_word)

In [12]:
for list_workds in textes['count_word'] :
    print()
    print(list_workds[:10])




[('technologies', 29.01449367737127), ('ai', 28.570848387080083), ('union', 24.67488938680225), ('data', 22.0925441097902), ('artificial', 18.811358135723644), ('intelligence', 18.699011117106956), ('related', 17.970288633686202), ('robotics', 16.55779951665001), ('development', 15.446260789180013), ('risk', 15.267423238946366)]

[('mec', 0.9072002232073403), ('event', 0.20362340776369134), ('wrap', 0.19700150832422172), ('booking', 0.13740441336899498), ('hover', 0.10429491617164678), ('colorskin', 0.10098396645191197), ('custom', 0.10098396645191197), ('calendar', 0.09270659215257493), ('color', 0.06621899439469638), ('shortcode', 0.059597094955226734)]

[('ai', 89.724927442238), ('work', 50.120828238949), ('data', 46.59643169296755), ('2021', 42.070300450868686), ('workers', 39.60754006052806), ('2020', 33.712572989348764), ('european', 31.644892542564577), ('systems', 28.819736133447027), ('management', 27.56040123114219), ('tools', 26.331377810816555)]

[('ai', 18.19835863882005)

# fichier csv

In [24]:
textes.to_csv('resultats_resume_tfidf.csv',sep=',')